# ESPI analyse - sample of 1000 transactions

## This jupyter notebook contains the processing and analysis of the ESPI reports in relation to the share price increase of the companies listed on the Warsaw Stock Exchange in the following steps:
1. Load information from all collected ESPI reports
2. Map ESPI report information with relevant symbol of the stocks
3. Load all GPW reports for all listed stocks
4. Define y, which is dependent on the trading result within the next days
5. Merge ESPI_info dataframe with stock quotes
6. Model definition and testing based on pyMorfologic tokenizer
7. Conclusions and recommendations

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

## 1. Load information from all collected ESPI reports

In [2]:
df =  pd.read_csv('espi_data.csv',dtype=str)
print(f'Shape of the dataframe: {df.shape}')
df.head(10)

Shape of the dataframe: (22333, 4)


,filename,date,name,file_content
0,./htmls/300000.html,20180130,BANK ZACHODNI WBK SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...
1,./htmls/300001.html,20180130,ZAKŁAD BUDOWY MASZYN ZREMB-CHOJNICE SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...
2,./htmls/300002.html,20180130,PRAGMA FAKTORING SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...
3,./htmls/300003.html,20180130,KREZUS SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...
4,./htmls/300004.html,NaN,MAKARONY POLSKIE SA,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
5,./htmls/300005.html,NaN,MBANK HIPOTECZNY,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
6,./htmls/300006.html,NaN,MBANK HIPOTECZNY,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
7,./htmls/300007.html,20180130,VOXEL SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...
8,./htmls/300008.html,20180130,POLSKIE GÓRNICTWO NAFTOWE I GAZOWNICTWO SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...
9,./htmls/300009.html,20180130,SWISSMED CENTRUM ZDROWIA SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...


In [3]:
df.describe()

,filename,date,name,file_content
count,22333,19945,22333,22333
unique,22333,602,616,22068
top,./htmls/330568.html,20180427,ULTIMATE GAMES SA,\n \n\n\n\n skorygowany\n \n \n \n \n \n \n \n...
freq,1,288,248,4


### Columns description:

* **filename** - Name of the html file of the ESPI report (also the name of the id used to scrape from Warsaw Stock Exchange website)
* **date** - Date of creation of the  the ESPI report ("Data sporządzenia"). It may be NaN in case where the date has not been identified in the file. <br>Those will not be taken fur further analysis (count of date field is lower than count of filename field)
* **name** - Name of the company traded on the Warsaw Stock Exchange
* **file_content** - Content of the ESPI report ("RAPORT BIEŻĄCY") used for NLP analysis


### Remove null values - due to null NaN values in date column

In [4]:
df.dropna(inplace=True)
df.shape

(19945, 4)

## 2. Map ESPI report information with relevant symbol of the stocks

The next step is to map the ESPI info with the symbol of the stocks from another source.
Some data may not be mapped as not all stocks are being quoted on the Warsaw Stock Exchange main index.

In [5]:
df_map = pd.read_csv('map_symbol-name.csv', sep='|')
df_map.Symbol=df_map.Symbol.str.lower()
print(f'Total number of mapped rows:',df_map.shape[0])
df_map.head()

Total number of mapped rows: 449


,Symbol,Nazwa
0,06n,MAGNA POLONIA SA
1,08n,OCTAVA SA
2,11b,11 BIT STUDIOS SA
3,1at,ATAL SA
4,4fm,4FUN MEDIA SA


### The two tables are merged based on the "name" and "Nazwa" columns

In [6]:
df = df.merge(df_map, how='left', left_on="name", right_on="Nazwa")
df.drop(columns=['Nazwa'], inplace=True)
print(f'Shape of the dataframe after mapping: {df.shape}')
df.head()

Shape of the dataframe after mapping: (19945, 5)


,filename,date,name,file_content,Symbol
0,./htmls/300000.html,20180130,BANK ZACHODNI WBK SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,NaN
1,./htmls/300001.html,20180130,ZAKŁAD BUDOWY MASZYN ZREMB-CHOJNICE SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,zre
2,./htmls/300002.html,20180130,PRAGMA FAKTORING SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,prf
3,./htmls/300003.html,20180130,KREZUS SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,kzs
4,./htmls/300007.html,20180130,VOXEL SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,vox


### Inspect the names that where not mapped
The reason for not beeing mapped may be the following:
* International companies reporting to the Warsaw Stock Exchange but no mapping is available / not being traded
* The ESPI reports from companies that are no longer traded
* Other name changes or naming convention issues that would need to be dealt with individually.

If the number of matches identified is not satisfactory, further rules on the data collection or name match should be introduced.

### Unmatched items from the ESPI reports table

In [7]:
matched_items=np.unique(df[~df.Symbol.isnull()].name)
print(f'Number of matched items:',len(matched_items))
unmatched_items=np.unique(df[df.Symbol.isnull()].name)
print(f'Number of unmatched items:',len(unmatched_items))

Number of matched items: 388
Number of unmatched items: 200


In [8]:
set(unmatched_items)

{'AAT HOLDING SA',
 'ABADON REAL ESTATE SA',
 'ABC DATA SA',
 'ABM SOLID SA',
 'AGROWILL GROUP AB',
 'AKCEPT FINANCE SA',
 'ALCHEMIA SA',
 'ALEJASAMOCHODOWA.PL SA',
 'ALMA MARKET SA',
 'AMBRA SA',
 'AMERICAN HEART OF POLAND SA',
 'APATOR SA',
 'AQUA SA',
 'AS SILVANO FASHION GROUP',
 'ASTARTA HOLDING N.V.',
 'ATLANTIS SA',
 'AUXILIA SA',
 'AVIA SOLUTIONS GROUP AB',
 'AVIAAM LEASING AB',
 'AWBUD SA',
 'BANCO SANTANDER SA',
 'BANK BGŻ BNP PARIBAS SA',
 'BANK GOSPODARSTWA KRAJOWEGO',
 'BANK ZACHODNI WBK SA',
 'BETA ETF MWIG40TR PORTFELOWY FUNDUSZ INWESTYCYJNY ZAMKNIĘTY',
 'BETA ETF WIG20LEV PORTFELOWY FUNDUSZ INWESTYCYJNY ZAMKNIĘTY',
 'BETA ETF WIG20SHORT PORTFELOWY FUNDUSZ INWESTYCYJNY ZAMKNIĘTY',
 'BETA ETF WIG20TR PORTFELOWY FUNDUSZ INWESTYCYJNY ZAMKNIĘTY',
 'BYTOM SA',
 'CASPAR ASSET MANAGEMENT SA',
 'CEZ, A.S.',
 'CITY SERVICE SE',
 'CLEAN&CARBON ENERGY SA',
 'COLIAN HOLDING SA',
 'CWA SA',
 'DIGITAL AVENUE SA',
 'E-STAR ALTERNATIV ENERGIASZOLGALTATO NYRT.',
 'ELKOP SA',
 'ELKOP SPÓŁ

### Unmatched items from the mapping table:

In [9]:
unmatched_mapping = set(df_map.Nazwa.unique())-set(df.name.unique())
pct_unmapped = round(100*len(unmatched_mapping)/len(set(df_map.Nazwa.unique())),2)
print(f'Unmatched items from the mapping table:',len(unmatched_mapping))
print(f'It constitues the following % of all mapped items:',pct_unmapped)
unmatched_mapping

Unmatched items from the mapping table: 61
It constitues the following % of all mapped items: 13.59


{'AGROTON GROUP OF COMPANIES',
 'ASTARTA HOLDING NV',
 'CEZ AS',
 'CITY SERVICE AS',
 'CLEAN & CARBON ENERGY SA',
 'COAL ENERGY SA',
 'DIGITREE GROUP SA',
 'DOM MAKLERSKI IDM SA',
 'ENEFI ENERGIAHATÉKONYSÁGI NYRT',
 'FABRYKA MASZYN FAMUR SA',
 'FABRYKA SPRZĘTU I NARZĘDZI GÓRNICZYCH FASING SA',
 'FENGHUA SOLETECH AG',
 'GRUPA AMBRA SA',
 'GRUPA APATOR SA',
 'GRUPA AZOTY POLICE SA',
 'GRUPO SANTANDER',
 'HERKULES SA',
 'IMMOFINANZ AG',
 'IMPERA CAPITAL ASI SA',
 'INTER RAO LIETUVA AB',
 'INTERNATIONAL PERSONAL FINANCE PLC',
 'INVESTMENT FRIENDS CAPITAL SE SA',
 'INVESTMENT FRIENDS SE SA',
 'IZOLACJA JAROCIN SA',
 'JJ AUTO AG',
 'JW CONSTRUCTION HOLDING SA',
 'KDM SHIPPING PUBLIC LIMITED',
 'KERNEL HOLDING SA',
 'KOSZALIŃSKIE PRZEDSIĘBIORSTWO PRZEMYSŁU DRZEWNEGO SA',
 'KRKA DD',
 'MARIE BRIZARDS WINE & SPIRITS SA',
 'MILKILAND NV',
 'MOL MAGYAR OLAJ-ÉS GÁZIPARI RÉSZVÉNYTÁRSASÁG',
 'MOSTOSTAL PLOCK SA',
 'MW TRADE SA',
 'NOVATURAS AB',
 'OVOSTAR UNION NV',
 'PA NOVA SA',
 'PARTNERBUD SA',


### The current number of matched items between ESPI Report and the mapping table is satisfactory to progress with further analysis.

## 3. Load all GPW reports for all listed stocks

The stock quote data is collected from the website: https://static.stooq.pl/db/h/d_pl_txt.zip
and was stored in *./GPW_stocks/GPW_stocks*

In [10]:
path = './GPW_stocks/GPW_stocks'
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.txt' in file:
            files.append(os.path.join(r, file))
files.sort()
akcje_all = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'OpenInt', 'Name'])

for f in files:
    try:
        # each txt file with quotes to be read into a dataframe and later combined
        akcje = pd.read_csv(f)
    except:
        print(f)
    # We are only interested in data from 2018 and 2019
    akcje_2018 = akcje[akcje["Date"]>=20180000].copy()
    akcje_2018["Name"]=f[len(path)+1:-4]
    akcje_all = akcje_all.append(akcje_2018,ignore_index=True)
print(f'Shape of the dataframe for stock quotes is: {akcje_all.shape}')
akcje_all.head()

Shape of the dataframe for stock quotes is: (180867, 8)


,Date,Open,High,Low,Close,Volume,OpenInt,Name
0,20180103,0.34,0.40,0.34,0.40,5740,0,06n
1,20180104,0.40,0.43,0.40,0.43,64407,0,06n
2,20180105,0.43,0.43,0.42,0.42,61414,0,06n
3,20180108,0.43,0.45,0.43,0.45,31870,0,06n
4,20180109,0.45,0.45,0.44,0.44,12989,0,06n


### The stock quotes table indicates the following information:
* **Date**: the date of the quote record
* **Open**: the opening price of the stock during the day
* **High**: the highest price of the stock during the day
* **Low**: the lowest price of the stock during the day
* **Close**: the closing price of the stock during the day
* **Volume**: volume of stocks being traded during the day
* **OpenInt**: unknown
* **Name**: symbol of the stock

### We are only interested in the "High" value of the stock trade

In [11]:
akcje_all = akcje_all.set_index([akcje_all.Name, akcje_all.Date])
akcje_all.drop(['Name','Date','OpenInt','Open','Low','Close','Volume'], axis=1, inplace=True)
akcje_all.head()

High
Name Date          
06n  20180103  0.40
     20180104  0.43
     20180105  0.43
     20180108  0.45
     20180109  0.45

## 4. Define y, which is dependent on the trading result within the next days

### First prepare the dataframe with the next days stock prices and dynamics

In [12]:
data = pd.concat(
            [akcje_all,
             -akcje_all.groupby('Name')['High'].diff(periods=-1).rename('High+1'),
             -akcje_all.groupby('Name')['High'].diff(periods=-2).rename('High+2'),
             -akcje_all.groupby('Name')['High'].diff(periods=-3).rename('High+3'),
             -akcje_all.groupby('Name')['High'].diff(periods=-4).rename('High+4'),
             -akcje_all.groupby('Name')['High'].diff(periods=-5).rename('High+5')],
           axis=1)
data.columns=['High','High+1','High+2','High+3','High+4','High+5']
data['High+1_pct']=data['High+1']/data['High']
data['High+2_pct']=data['High+2']/data['High']
data['High+3_pct']=data['High+3']/data['High']
data['High+4_pct']=data['High+4']/data['High']
data['High+5_pct']=data['High+5']/data['High']
data = data.reset_index()
data.head()

,Name,Date,High,High+1,High+2,High+3,High+4,High+5,High+1_pct,High+2_pct,High+3_pct,High+4_pct,High+5_pct
0,06n,20180103,0.40,0.03,0.03,0.05,0.05,-0.00,0.075000,0.075000,0.125000,0.125000,-0.000000
1,06n,20180104,0.43,-0.00,0.02,0.02,-0.03,-0.05,-0.000000,0.046512,0.046512,-0.069767,-0.116279
2,06n,20180105,0.43,0.02,0.02,-0.03,-0.05,-0.05,0.046512,0.046512,-0.069767,-0.116279,-0.116279
3,06n,20180108,0.45,-0.00,-0.05,-0.07,-0.07,-0.02,-0.000000,-0.111111,-0.155556,-0.155556,-0.044444
4,06n,20180109,0.45,-0.05,-0.07,-0.07,-0.02,-0.03,-0.111111,-0.155556,-0.155556,-0.044444,-0.066667


* **High+x** indicates the price of the period +1, +2, +3, etc
* **High+x_pct** indicates the price change compared to High (period 0)

### Set y=1 when criteria is met, eg. when any of the next 2 days' quote >= 3% than period 0

*Other scenarios may include the whole 5 day period or other combinations how y is calculated (beyond the scope of current project).*

*In particular y can also be calculated in multinomial variation, e.g.:*
* **y=0** where the stock price dynamics has been low in the period analysed
* **y=1** where the stock price increases significantly in the period analysed
* **y=2** where the stock price decreases significantly in the period analysed (in analysing the results it would be important to limit this category in measuring the results)

In [13]:
k=0.03
data["y"]=pd.DataFrame([data["High+1_pct"]>=k, data["High+2_pct"]>=k]).any().astype(int)           
data.head()

,Name,Date,High,High+1,High+2,High+3,High+4,High+5,High+1_pct,High+2_pct,High+3_pct,High+4_pct,High+5_pct,y
0,06n,20180103,0.40,0.03,0.03,0.05,0.05,-0.00,0.075000,0.075000,0.125000,0.125000,-0.000000,1
1,06n,20180104,0.43,-0.00,0.02,0.02,-0.03,-0.05,-0.000000,0.046512,0.046512,-0.069767,-0.116279,1
2,06n,20180105,0.43,0.02,0.02,-0.03,-0.05,-0.05,0.046512,0.046512,-0.069767,-0.116279,-0.116279,1
3,06n,20180108,0.45,-0.00,-0.05,-0.07,-0.07,-0.02,-0.000000,-0.111111,-0.155556,-0.155556,-0.044444,0
4,06n,20180109,0.45,-0.05,-0.07,-0.07,-0.02,-0.03,-0.111111,-0.155556,-0.155556,-0.044444,-0.066667,0


### Check the share of the population subject to the criteria set compared to the whole data set to see if is is highly unbalanced (eg. y=1 would be less then 10% of the population).

In [14]:
data.y.sum()/data.shape[0]

0.15976933326698625

## 5. Merge ESPI_info dataframe with stock quotes

### Create an id to concatenate ESPI_info data frame with quotes data frame. The id is created by concatenating the symbol with the date in both tables.

**Remark 1:**
*There may be cases of multiple ESPI reports submitted during one trading day. This will result with creating ids that are not unique. When the data frames are merged, those should result in multiple links per one id and is expected. The difficulty though is that two reports will be used to predict the same change in stock prices, where the model will be used two reports combined on the change of the stock price, although the reports may in fact impact the price in an opposite way. In this example, such cases are not reviewed in detail in further processing.*

**Remark 2:**
*In this approach - we only take into account the cases when the ESPI report is submitted on the date of trading (then the name of the company and date will match the trading information of the same day). However, there are situations where the ESPI report is submitted before the trading day - and such cases will not be identified  here. These cases might be identified and analysed by building additional functions (beyond the scope of this project).*

In [15]:
data["id"]=data.Name+data.Date.astype(str)
data = data[['id','y']]
data.head()

,id,y
0,06n20180103,1
1,06n20180104,1
2,06n20180105,1
3,06n20180108,0
4,06n20180109,0


In [16]:
df["id"]=df.Symbol+df.date.astype(str)
df.dropna(inplace=True)
df.head()

,filename,date,name,file_content,Symbol,id
1,./htmls/300001.html,20180130,ZAKŁAD BUDOWY MASZYN ZREMB-CHOJNICE SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,zre,zre20180130
2,./htmls/300002.html,20180130,PRAGMA FAKTORING SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,prf,prf20180130
3,./htmls/300003.html,20180130,KREZUS SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,kzs,kzs20180130
4,./htmls/300007.html,20180130,VOXEL SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,vox,vox20180130
5,./htmls/300008.html,20180130,POLSKIE GÓRNICTWO NAFTOWE I GAZOWNICTWO SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,pgn,pgn20180130


In [17]:
df1 = df.merge(data,how='left',on='id')
print(f'After merging the dataframe received the shape: {df1.shape}')
df1.head()

After merging the dataframe received the shape: (16331, 7)


,filename,date,name,file_content,Symbol,id,y
0,./htmls/300001.html,20180130,ZAKŁAD BUDOWY MASZYN ZREMB-CHOJNICE SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,zre,zre20180130,0.0
1,./htmls/300002.html,20180130,PRAGMA FAKTORING SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,prf,prf20180130,0.0
2,./htmls/300003.html,20180130,KREZUS SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,kzs,kzs20180130,0.0
3,./htmls/300007.html,20180130,VOXEL SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,vox,vox20180130,0.0
4,./htmls/300008.html,20180130,POLSKIE GÓRNICTWO NAFTOWE I GAZOWNICTWO SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,pgn,pgn20180130,0.0


### Remove nulls where ESPI_info has not been merged with stocks

In [18]:
df = df1[~df1.y.isnull()].reset_index()

### Number of y=1 in the population - when low - than we should think of balancing the population for further modelling

In [19]:
df.y.mean()

0.1833861316437874

### With this example the 18% of y in the population seems satisfactory for further processing.

In [20]:
df.shape

(14205, 8)

In [21]:
df.y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: y, dtype: float64

In [22]:
y = df.y

In [23]:
df.file_content.head()

0    \n \n komisja nadzoru finansowego\n \n\n \n\n\...
1    \n \n komisja nadzoru finansowego\n \n\n \n\n\...
2    \n \n komisja nadzoru finansowego\n \n\n \n\n\...
3    \n \n komisja nadzoru finansowego\n \n\n \n\n\...
4    \n \n komisja nadzoru finansowego\n \n\n \n\n\...
Name: file_content, dtype: object

## 6. Model definition and testing based on pyMorfologic tokenizer

In [24]:
import nltk 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from warnings import filterwarnings
filterwarnings("ignore")

### Tokenize the text field using nltk

## Select the sample of y and X by balancing and limiting of the population to 1000: select 50% of y = 1 and 50% of y = 0

In [39]:
part_a = df[df.y==1].sample(n=500)
part_b = df[df.y==0].sample(n=500)
df = pd.concat([part_a, part_b], ignore_index=True)

In [40]:
df['tokenized_text'] = df['file_content'].apply(nltk.word_tokenize) 

In [41]:
df.head()

,index,filename,date,name,file_content,Symbol,id,y,tokenized_text
0,6967,./htmls/315135.html,20180912,LENA LIGHTING SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,len,len20180912,1.0,"[komisja, nadzoru, finansowego, raport, bieżąc..."
1,9356,./htmls/331944.html,20190605,CERAMIKA NOWA GALA SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,cng,cng20190605,1.0,"[komisja, nadzoru, finansowego, raport, bieżąc..."
2,13983,./htmls/338163.html,20190916,GETIN HOLDING SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,gtn,gtn20190916,1.0,"[komisja, nadzoru, finansowego, raport, bieżąc..."
3,7293,./htmls/320018.html,20181203,POLSKI HOLDING NIERUCHOMOŚCI SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,phn,phn20181203,1.0,"[komisja, nadzoru, finansowego, raport, bieżąc..."
4,8541,./htmls/330368.html,20190528,BANK MILLENNIUM SA,\n \n komisja nadzoru finansowego\n \n\n \n\n\...,mil,mil20190528,1.0,"[komisja, nadzoru, finansowego, raport, bieżąc..."


### Use pyMorfologik to parse Polish stemming in the ESPI reports.

In [42]:
from pyMorfologik import Morfologik
from pyMorfologik.parsing import ListParser

In [43]:
parser = ListParser()
stemmer = Morfologik()

### The stemming function will use the first stemming option from the available list of stems for selected word

In [44]:
def stemming_func(tokenized_t):
    stemmed_text = stemmer.stem(tokenized_t, parser)
    return [list(stemmed_text[i][1].keys())[0] for i in range(len(stemmed_text)) if len(list(stemmed_text[i][1].keys()))>0]

In [45]:
# example of stemming
stemming_func(df.tokenized_text.loc[0])

['komisja',
 'nadzór',
 'finansowy',
 'raport',
 'bieżący',
 'numer',
 'data',
 'sporządzić',
 'skrócić',
 'nazwa',
 'emitent',
 'temat',
 'powiadomienie',
 'o',
 'transakcja',
 'nabycie',
 'akcja',
 'przez',
 'członek',
 'zarząd',
 'spółka',
 'artykuł',
 'mara',
 'podstawa',
 'prawny',
 'artykuł',
 'usta',
 'mara',
 'informacja',
 'o',
 'transakcja',
 'wykonywać',
 'przez',
 'osoba',
 'pełnić',
 'obowiązek',
 'zarządczy',
 'treść',
 'raport',
 'spółka',
 'z',
 'siedziba',
 'w',
 'środa',
 'wielkopolski',
 'informować',
 'że',
 'w',
 'dzień',
 'rok',
 'otrzymać',
 'powiadomienie',
 'od',
 'osoba',
 'pełnić',
 'funkcja',
 'zarządczy',
 'w',
 'spółka',
 'pan',
 'filipiński',
 'członek',
 'zarząd',
 'spółka',
 'o',
 'nabycie',
 'akcja',
 'w',
 'tryb',
 'artykuł',
 'mara',
 'spółka',
 'w',
 'załączenie',
 'przekazywać',
 'treść',
 'powiadomienie',
 'załącznik',
 'plik',
 'opis',
 'zawiadomienie',
 'o',
 'nabycie',
 'akcja',
 'circa',
 'powiadomienie',
 'o',
 'nabycie',
 'akcja']

**Remark:** The following code  works very slowly as the pyMorfologic parser works with lists and not dataframes

In [46]:
start_time = datetime.now()
df['text_lemma'] = df['tokenized_text'].map(stemming_func) 
end_time = datetime.now()
time_taken = end_time-start_time
print(f'The time it took to process stemming on all items: {time_taken}')

The time it took to process stemming on all items: 0:02:44.633224


In [47]:
len(df.text_lemma.iloc[0])

89

### Set up stop words list, by also adding letters from the alphabet

In [48]:
from stop_words import get_stop_words
stop_words = get_stop_words('pl')
other_stop_words = list('abcdefghijklmnopqrstuvxwyz')
stop_words.extend(other_stop_words)
stop_words=sorted(list(set(stop_words)))
stop_words

['a',
 'ach',
 'aj',
 'albo',
 'b',
 'bardzo',
 'bez',
 'bo',
 'być',
 'c',
 'ci',
 'ciebie',
 'cię',
 'co',
 'czy',
 'd',
 'daleko',
 'dla',
 'dlaczego',
 'dlatego',
 'do',
 'dobrze',
 'dokąd',
 'dość',
 'dużo',
 'dwa',
 'dwaj',
 'dwie',
 'dwoje',
 'dzisiaj',
 'dziś',
 'e',
 'f',
 'g',
 'gdyby',
 'gdzie',
 'go',
 'h',
 'i',
 'ich',
 'ile',
 'im',
 'inny',
 'j',
 'ja',
 'jak',
 'jakby',
 'jaki',
 'je',
 'jeden',
 'jedna',
 'jedno',
 'jego',
 'jej',
 'jemu',
 'jest',
 'jestem',
 'jeśli',
 'jeżeli',
 'już',
 'ją',
 'k',
 'każdy',
 'kiedy',
 'kierunku',
 'kto',
 'ku',
 'l',
 'lub',
 'm',
 'ma',
 'mają',
 'mam',
 'mi',
 'mnie',
 'mną',
 'moi',
 'moja',
 'moje',
 'może',
 'mu',
 'my',
 'mój',
 'n',
 'na',
 'nam',
 'nami',
 'nas',
 'nasi',
 'nasz',
 'nasza',
 'nasze',
 'natychmiast',
 'nic',
 'nich',
 'nie',
 'niego',
 'niej',
 'niemu',
 'nigdy',
 'nim',
 'nimi',
 'nią',
 'niż',
 'o',
 'obok',
 'od',
 'około',
 'on',
 'ona',
 'one',
 'oni',
 'ono',
 'owszem',
 'p',
 'po',
 'pod',
 'ponieważ'

In [49]:
def remove_stop_words(lemma_text):
    stop_words_removed = [a for a in lemma_text if a not in stop_words]  
    return " ".join(stop_words_removed)   

In [50]:
# example of removing stop words
example_report = df.text_lemma.iloc[0]
remove_stop_words(example_report)

'komisja nadzór finansowy raport bieżący numer data sporządzić skrócić nazwa emitent temat powiadomienie transakcja nabycie akcja przez członek zarząd spółka artykuł mara podstawa prawny artykuł usta mara informacja transakcja wykonywać przez osoba pełnić obowiązek zarządczy treść raport spółka siedziba środa wielkopolski informować dzień rok otrzymać powiadomienie osoba pełnić funkcja zarządczy spółka pan filipiński członek zarząd spółka nabycie akcja tryb artykuł mara spółka załączenie przekazywać treść powiadomienie załącznik plik opis zawiadomienie nabycie akcja circa powiadomienie nabycie akcja'

In [51]:
df['text_mod'] = df['text_lemma'].apply(remove_stop_words) 

In [52]:
X = df.text_mod
X.head()

0    komisja nadzór finansowy raport bieżący numer ...
1    komisja nadzór finansowy raport bieżący numer ...
2    komisja nadzór finansowy raport bieżący numer ...
3    komisja nadzór finansowy raport bieżący numer ...
4    komisja nadzór finansowy raport bieżący numer ...
Name: text_mod, dtype: object

In [53]:
# example of the X, y record and number of X records in the population.
i=100
print(f'Position {i} results in \ny = {y.iloc[i]}')
print(f'and X = \n{X.iloc[i]}')
print(f'Shape of the table is {X.shape}')

Position 100 results in 
y = 0.0
and X = 
komisja nadzór finansowy raport bieżący numer data sporządzić skrócić nazwa emitent temat żądanie zwołanie nadzwyczajny walny zgromadzenie podstawa prawny artykuł usta punkt ustawa oferta informacja bieżący okresowy treść raport zarząd siedziba poznanie spółka informować dzień czerwiec godzina popołudniowy otrzymać powszechny towarzystwo emerytalny działać imienie otwarty fundusz emerytalny akcjonariusz zgłosić podstawa artykuł kodeks spółka handlowy żądanie zwołanie dzień lipiec nadzwyczajny walny zgromadzenie spółka następujący porządek obrada otwarcie walny wybór przewodniczący walny stwierdzenie prawidłowość zwołanie walny zgromadzenie zdolności podejmować przyjęcie porządek podjęcie uchwała sprawa zmiana skład rad nadzorczy spółka zamknięcie obrada treść projekt uchwała wraz uzasadnienie zgłosić przez akcjonariusz spółka przekazywać załącznik niniejszy raport bieżący dodatkowo spółka dołączać informacja na temat kandydat niezależny członek

In [54]:
cv = CountVectorizer(min_df=10, max_df=0.9)
cv.fit(X)
X_cv=cv.transform(X)
X_cv

<1000x1167 sparse matrix of type '<class 'numpy.int64'>'
	with 70158 stored elements in Compressed Sparse Row format>

In [55]:
# Top 5 words
k = 5 
for i in range(10):
    order = [X_cv[i].todense().argsort().A.flatten()]
    print(np.array(cv.get_feature_names())[order][-k:][::-1])

['spółka' 'nabycie' 'akcja' 'powiadomienie' 'mara']
['informacja' 'dzień' 'oraz' 'fundusz' 'akcjonariusz']
['dzień' 'informacja' 'okresowy' 'publiczny' 'września']
['seria' 'spółka' 'dzień' 'akcja' 'grudzień']
['bank' 'euro' 'akcja' 'millennium' 'nabycie']
['akcja' 'własny' 'nabywać' 'spółka' 'nabycie']
['głos' 'liczba' 'oraz' 'ogólny' 'stanowić']
['głos' 'zgromadzenie' 'liczba' 'walny' 'udział']
['czerwiec' 'rok' 'sprzedaż' 'milion' 'złoty']
['rok' 'według' 'warszawa' 'sprawa' 'handlowy']


In [56]:
len(cv.get_feature_names())

1167

### It is good to identify the set of words applicable to y=1 population.

In [61]:
y=df.y
C_index = y[y>0].index
C_index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            490, 491, 492, 493, 494, 495, 496, 497, 498, 499],
           dtype='int64', length=500)

In [62]:
X_indexed = X.loc[C_index]

In [63]:
cv = CountVectorizer(min_df=10, max_df=0.9)
cv.fit(X_indexed)
X_indexed_cv=cv.transform(X_indexed)

In [64]:
len(cv.get_feature_names())

762

In [65]:
important_set = set(np.array(cv.get_feature_names()))

In [66]:
important_set

{'absolutorium',
 'adres',
 'akcja',
 'akcjonariusz',
 'akcyjny',
 'aktualny',
 'aktyw',
 'aleja',
 'alternatywny',
 'analogiczny',
 'andante',
 'aneks',
 'ani',
 'artykuł',
 'audyt',
 'audytorski',
 'badanie',
 'bank',
 'bankowy',
 'bar',
 'bezpośrednio',
 'biegły',
 'biuro',
 'bliski',
 'blisko',
 'brak',
 'branża',
 'brutto',
 'budowa',
 'budowlany',
 'budynek',
 'była',
 'były',
 'całość',
 'cały',
 'cel',
 'cena',
 'centrum',
 'charakter',
 'chwila',
 'chyba',
 'cywilny',
 'czas',
 'czerwiec',
 'czterdzieści',
 'cztery',
 'czterysta',
 'czym',
 'czynności',
 'część',
 'członek',
 'członkowski',
 'dalej',
 'daleki',
 'dane',
 'dany',
 'dawać',
 'decyzja',
 'depozyt',
 'dodatkowo',
 'dodatkowy',
 'dokonanie',
 'dokonany',
 'dokonać',
 'dokonywać',
 'dokument',
 'dokumentacja',
 'dom',
 'dopuścić',
 'doradca',
 'dostępny',
 'dotychczas',
 'dotychczasowy',
 'dotyczy',
 'dotyczyć',
 'droga',
 'drugi',
 'dwadzieścia',
 'dwieście',
 'dyrektor',
 'dyrektywa',
 'dywidenda',
 'działalność',

In [67]:
cv = CountVectorizer(min_df=10, max_df=0.9)
cv.fit(X)
X_cv=cv.transform(X)

### Set up a different set of stop words, that do not contain important words from the y=1 population.

In [68]:
stop_words_moje = set(np.array(cv.get_feature_names())) - important_set

In [69]:
len(stop_words_moje)

405

In [70]:
stop_words_moje

{'absolwent',
 'administracja',
 'agencja',
 'aktualizacja',
 'aktualnie',
 'analiza',
 'animator',
 'as',
 'at',
 'azot',
 'bilans',
 'biznes',
 'biznesowy',
 'brzmienie',
 'budownictwo',
 'business',
 'by',
 'bądź',
 'całkowity',
 'certyfikat',
 'ciąg',
 'czego',
 'datować',
 'deklaracja',
 'delegowany',
 'deweloperski',
 'do',
 'dobry',
 'dochodzenie',
 'dojść',
 'dominować',
 'doradztwo',
 'dostarczyć',
 'dostawa',
 'dostęp',
 'dołączyć',
 'doświadczenie',
 'duży',
 'dyspozycja',
 'dystrybucja',
 'działka',
 'dziesięć',
 'dziewięćdziesiąt',
 'długoterminowy',
 'efekt',
 'egzekucja',
 'ekonomiczny',
 'elektryczny',
 'element',
 'energia',
 'europ',
 'ewentualny',
 'faktoring',
 'figurować',
 'for',
 'forma',
 'gdański',
 'generalny',
 'granica',
 'grosz',
 'gwarancyjny',
 'harmonogram',
 'hipoteka',
 'hotel',
 'identyfikacja',
 'ilości',
 'ilość',
 'imienny',
 'info',
 'informacyjny',
 'inwestorski',
 'inżynier',
 'jednostki',
 'jedyny',
 'kandydat',
 'kandydatura',
 'kary',
 'kasa'

In [71]:
X.shape

(1000,)

### Run Grid Search on count vectorizer processing and models with various parameters.

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=100)

pipelines = [Pipeline([
    ('vect', CountVectorizer()),
    ('lr', LogisticRegression()),
]),
             Pipeline([
    ('vect', CountVectorizer()),
    ('lda', LatentDirichletAllocation()),
    ('lr', LogisticRegression()),
]),
             
             Pipeline([
    ('vect', CountVectorizer()),
    ('tree', DecisionTreeClassifier()),
]),
             Pipeline([
    ('vect', CountVectorizer()),
    ('forest', RandomForestClassifier()),
])
]

parameters = [
    [
    {
        'vect__stop_words': (stop_words,stop_words_moje),
        'vect__min_df': (10, 100, 1000),
        'vect__max_df': (0.5, 0.75),
        'lr__C': (0.01, 1, 10),
    }
],
        [
    {
        'vect__stop_words': (stop_words,stop_words_moje),
        'vect__min_df': (10, 100, 1000),
        'vect__max_df': (0.5, 0.75),
        'lda__n_components': (10, 20, 40),
        'lr__C': (0.01, 1, 10),
    }
],
        [
    {
        'vect__stop_words': (stop_words,stop_words_moje),
        'vect__min_df': (10, 100, 1000),
        'vect__max_df': (0.5, 0.75),
        'tree__min_samples_leaf': (10, 100),
    }
],
        [
    {
        'vect__stop_words': (stop_words,stop_words_moje),
        'vect__min_df': (10, 100, 1000),
        'vect__max_df': (0.5, 0.75),
        'forest__min_samples_leaf': (10, 100),
        'forest__n_estimators': (10, 100, 500),
    }
]
]

models=[]
for pipeline, parameter in zip(pipelines,parameters):
    gs = GridSearchCV(pipeline, parameter, cv=10, n_jobs=-1)
    gs.fit(X_train, y_train)
    models.append(gs)
    predictions = gs.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
    print('\n')
    print(str(type(gs.estimator))[16:-2])
    print(gs.best_params_)
    print(f'ACCURACY SCORE:',accuracy_score(y_test, predictions))
    print(classification_report(y_test, predictions))
    print(f'tn, fp, fn, tp:',(tn, fp, fn, tp))
    print(f'All y=1 predictions:',predictions.sum())



pipeline.Pipeline
{'lr__C': 1, 'vect__max_df': 0.75, 'vect__min_df': 10, 'vect__stop_words': ['a', 'ach', 'aj', 'albo', 'b', 'bardzo', 'bez', 'bo', 'być', 'c', 'ci', 'ciebie', 'cię', 'co', 'czy', 'd', 'daleko', 'dla', 'dlaczego', 'dlatego', 'do', 'dobrze', 'dokąd', 'dość', 'dużo', 'dwa', 'dwaj', 'dwie', 'dwoje', 'dzisiaj', 'dziś', 'e', 'f', 'g', 'gdyby', 'gdzie', 'go', 'h', 'i', 'ich', 'ile', 'im', 'inny', 'j', 'ja', 'jak', 'jakby', 'jaki', 'je', 'jeden', 'jedna', 'jedno', 'jego', 'jej', 'jemu', 'jest', 'jestem', 'jeśli', 'jeżeli', 'już', 'ją', 'k', 'każdy', 'kiedy', 'kierunku', 'kto', 'ku', 'l', 'lub', 'm', 'ma', 'mają', 'mam', 'mi', 'mnie', 'mną', 'moi', 'moja', 'moje', 'może', 'mu', 'my', 'mój', 'n', 'na', 'nam', 'nami', 'nas', 'nasi', 'nasz', 'nasza', 'nasze', 'natychmiast', 'nic', 'nich', 'nie', 'niego', 'niej', 'niemu', 'nigdy', 'nim', 'nimi', 'nią', 'niż', 'o', 'obok', 'od', 'około', 'on', 'ona', 'one', 'oni', 'ono', 'owszem', 'p', 'po', 'pod', 'ponieważ', 'przed', 'przedtem',



pipeline.Pipeline
{'forest__min_samples_leaf': 10, 'forest__n_estimators': 10, 'vect__max_df': 0.75, 'vect__min_df': 100, 'vect__stop_words': {'oddać', 'składanie', 'prawie', 'przyznać', 'zmniejszyć', 'poddanie', 'for', 'występować', 'negocjacja', 'dyspozycja', 'którykolwiek', 'konieczny', 'wypłacić', 'wejść', 'kwartalne', 'wielkopolski', 'wymóg', 'sieć', 'wpływać', 'technologia', 'wiedza', 'względem', 'ilość', 'towarowy', 'zgodny', 'przygotowanie', 'at', 'osobisty', 'użytkowanie', 'naruszenie', 'ukończyć', 'szkoła', 'pani', 'jednostki', 'uprawnienie', 'zaliczka', 'zlecenie', 'płatność', 'poczet', 'przenieść', 'oferować', 'harmonogram', 'gwarancyjny', 'do', 'budownictwo', 'dystrybucja', 'elektryczny', 'osobiście', 'ważność', 'report', 'środowisko', 'park', 'maksymalny', 'hipoteka', 'roboczy', 'certyfikat', 'sprzedający', 'ład', 'zbyć', 'korporacyjny', 'dostarczyć', 'bilans', 'sesja', 'trwać', 'koszty', 'zmierzać', 'jedyny', 'pozytywnie', 'przedwstępny', 'no', 'millennium', 'płatny', 

## 7. Conclusions and recommendations

The results of the modelling on a limited and balanced sample are still unsatisfactory, with the low accuracy levels and the precision metrics below 50% (very unsatisfactory). 

Other models and preprocessing may be used for better accuracy and precision levels (e.g. Tfidfvectorizer for preprocessing / decomposition; and XGBoost or neural networks for modelling) - out of scope of this project.